In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

# Context

In [18]:
df = pd.read_csv("train.csv", index_col="PassengerId")
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
0,1,1,"Oconnor, Frankie",male,NaN,2,0,209245,27.14,C12239,S
1,0,3,"Bryan, Drew",male,NaN,0,0,27323,13.35,NaN,S
2,0,3,"Owens, Kenneth",male,0.33,1,2,CA 457703,71.29,NaN,S
3,0,3,"Kramer, James",male,19.00,0,0,A. 10866,13.04,NaN,S
4,1,3,"Bond, Michael",male,25.00,0,0,427635,7.76,NaN,S


# Data quality assessment

In [3]:
# Get the overall description of the df: rows, columns, data types, count of non-null values for each column
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 11 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Survived  100000 non-null  int64  
 1   Pclass    100000 non-null  int64  
 2   Name      100000 non-null  object 
 3   Sex       100000 non-null  object 
 4   Age       96708 non-null   float64
 5   SibSp     100000 non-null  int64  
 6   Parch     100000 non-null  int64  
 7   Ticket    95377 non-null   object 
 8   Fare      99866 non-null   float64
 9   Cabin     32134 non-null   object 
 10  Embarked  99750 non-null   object 
dtypes: float64(2), int64(4), object(5)
memory usage: 9.2+ MB


Columns with missing values: Age, Fare, Embarked, Cabin, Ticket. Cabin column contains ~70% of Null values.
Data types are correct.

In [ ]:
# Check on full duplicates (excluding generated PassengerId)
dupl = df[["Survived", "Pclass", "Name", "Sex", "Age", "SibSp", "Parch", "Ticket", "Cabin", "Fare", "Embarked"]].duplicated(keep=False).reset_index().rename(columns={0:'duplicate'})
dupl[dupl["duplicate"]==True]["duplicate"].count()

In [19]:
# Columns are supposed to be useless for data analysis 
# Cabin - the majority of values are NaN
# Ticket - technical values without business sense
# Name - PII data without any value for data analysis
useless_columns = ["Name", "Ticket", "Cabin"] 
df = df.drop(columns=useless_columns) # Drop useless columns from the df to improve performance while df processing
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
0,1,1,male,NaN,2,0,27.14,S
1,0,3,male,NaN,0,0,13.35,S
2,0,3,male,0.33,1,2,71.29,S
3,0,3,male,19.00,0,0,13.04,S
4,1,3,male,25.00,0,0,7.76,S


In [6]:
# Get an overall summary of the dataset using descriptive statistics of numeric attributes
df.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,100000.000000,100000.000000,96708.000000,100000.000000,100000.000000,99866.00000
mean,0.427740,2.106910,38.355472,0.397690,0.454560,43.92933
std,0.494753,0.837727,18.313556,0.862566,0.950076,69.58882
min,0.000000,1.000000,0.080000,0.000000,0.000000,0.68000
25%,0.000000,1.000000,25.000000,0.000000,0.000000,10.04000
50%,0.000000,2.000000,39.000000,0.000000,0.000000,24.46000
75%,1.000000,3.000000,53.000000,1.000000,1.000000,33.50000
max,1.000000,3.000000,87.000000,8.000000,9.000000,744.66000


Value range for each column looks meaningful and correct based on min, max and mean values.

In [5]:
# Check for unique values per column
val_counts_dict = {}
for column in df.columns:
    val_counts_dict[column] = df[column].value_counts().to_dict()
val_counts_dict

{'Survived': {0: 57226, 1: 42774},
 'Pclass': {3: 41006, 1: 30315, 2: 28679},
 'Sex': {'male': 56114, 'female': 43886},
 'Age': {25.0: 2354,
  23.0: 2345,
  26.0: 2320,
  24.0: 2298,
  27.0: 2088,
  22.0: 2019,
  52.0: 1924,
  28.0: 1911,
  53.0: 1905,
  55.0: 1840,
  51.0: 1836,
  56.0: 1825,
  54.0: 1819,
  57.0: 1803,
  29.0: 1769,
  49.0: 1748,
  50.0: 1740,
  58.0: 1712,
  48.0: 1701,
  21.0: 1675,
  30.0: 1661,
  34.0: 1649,
  31.0: 1631,
  59.0: 1620,
  47.0: 1616,
  46.0: 1610,
  36.0: 1608,
  32.0: 1574,
  60.0: 1550,
  38.0: 1547,
  33.0: 1529,
  35.0: 1528,
  45.0: 1525,
  40.0: 1522,
  37.0: 1504,
  41.0: 1497,
  61.0: 1496,
  39.0: 1479,
  43.0: 1478,
  44.0: 1477,
  42.0: 1425,
  20.0: 1387,
  62.0: 1361,
  63.0: 1253,
  64.0: 1102,
  65.0: 1069,
  4.0: 1012,
  5.0: 1008,
  6.0: 970,
  19.0: 970,
  66.0: 942,
  7.0: 941,
  3.0: 925,
  2.0: 879,
  8.0: 816,
  67.0: 768,
  9.0: 730,
  18.0: 716,
  68.0: 678,
  1.0: 649,
  10.0: 602,
  69.0: 561,
  11.0: 517,
  70.0: 465,
  

In [20]:
# Fill Null values
# Age - replace NULLs with mean value
df['Age'] = df['Age'].fillna(df['Age'].median())

# Fare - replace NULLs with mean value depending on the passenger's class
cond1 = (df['Pclass'] == 1)
cond2 = (df['Pclass'] == 2)
cond3 = (df['Pclass'] == 3)
df.loc[cond1,'Fare'] = df.loc[cond1,'Fare'].fillna(df.loc[cond1,'Fare'].mean())
df.loc[cond2,'Fare'] = df.loc[cond2,'Fare'].fillna(df.loc[cond2,'Fare'].mean())
df.loc[cond3,'Fare'] = df.loc[cond3,'Fare'].fillna(df.loc[cond3,'Fare'].mean())

# Embarked - replace NULLs with mode value (the most frequent)
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])

# Data exploration

In [ ]:
sns.catplot('Sex', data=df, hue='Survived', kind='count')

In [ ]:
sns.catplot('Pclass', data=df, hue='Survived', kind='count')

In [ ]:
sns.catplot('Parch', data=df, hue='Survived', kind='count')

# Summary